# 2. Working With Pinecone Vector Database
Although Vector Databases have existed long before Large Language Models, vector DBs have become an important part of many LLM solutions. In particular, Retreival Augmented Generation (or RAG) architecture addresses LLM's halucinations and issues with longer-term memory by augmenting the user's prompt with the results of a search accross a vector DB. [Pinecone](https://www.pinecone.io/) is a cloud-based vector database that is easy to integrate with your Cloudera AI workflow, as this notebook shows. 

We pre-loaded the Pinecone Vector DB using Cloudera AI's Jobs feature that scraped a few sample policies, cleaned up the HTML tags and loaded each page into Pinecone. This notebook will focus on interacting with Pinecone from a Jupyter notebook.

![Exercise 2 overview](../assets/exercise_2.png)

### 2.1 Imports and global vars

In [1]:
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX = os.getenv('PINECONE_INDEX')
dimension = 768

/home/cdsw/.local/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 2.2 Initialize Pinecone connection
Pinecone client is initialized with the parameters defined previously. 

In [2]:
print("initialising Pinecone connection...")
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    
print("Pinecone initialised")

print(f"Getting '{PINECONE_INDEX}' as object...")
index = pc.Index(PINECONE_INDEX)
print("Success")

# Get latest statistics from index
current_collection_stats = index.describe_index_stats()
print('Total number of embeddings in Pinecone index is {}.'.format(current_collection_stats.get('total_vector_count')))

initialising Pinecone connection...
Pinecone initialised
Getting 'ecss-docs' as object...
Success
Total number of embeddings in Pinecone index is 3.


### 2.3 Function to peform the vector search 
The idea is to find a chunk from the Knowledge Base that is "close" to what the original user's prompt is. We perform a semantic search using the user's question, find the nearest knowledge base chunk, and return the content of that chunk along with its source and score.

In [3]:
# Get embeddings for a user question and query Pinecone vector DB for nearest knowledge base chunk
def get_nearest_chunk_from_pinecone_vectordb(index, question):
    # Generate embedding for user question with embedding model
    retriever = SentenceTransformer(EMBEDDING_MODEL_REPO)
    xq = retriever.encode([question]).tolist()
    xc = index.query(vector=xq, top_k=5,
                 include_metadata=True)
    
    matching_files = []
    scores = []
    for match in xc['matches']:
        # extract the 'file_path' within 'metadata'
        file_path = match['metadata']['file_path']
        # extract the individual scores for each vector
        score = match['score']
        scores.append(score)
        matching_files.append(file_path)

    # Return text of the nearest knowledge base chunk 
    # Note that this ONLY uses the first matching document for semantic search. matching_files holds the top results so you can increase this if desired.
    response = load_context_chunk_from_data(matching_files[0])
    sources = matching_files[0]
    score = scores[0]
    return response, sources, score

# Return the Knowledge Base doc based on Knowledge Base ID (relative file path)
def load_context_chunk_from_data(id_path):
    with open(id_path, "r") as f: # Open file in read mode
        return f.read()

### 2.4 Examine the results of the vector search
Given the text of the question, we can now perform a vector search and output the results in the notebook. An important detail here is the ability to interact with metadata (e.g. context source) which can be used to narrow down the search space and, more critically, for authorization. These approaches are out of scope of this lab. 

In [4]:
question = "What is your return policy?" ## (Swap with your own based on your dataset)

context_chunk, sources, score = get_nearest_chunk_from_pinecone_vectordb(index, question)
print("\nContext Chunk: ")
print(context_chunk)
print("\nContext Source(s): ")
print(sources)
print("\nPinecone Score: ")
print(score)


Context Chunk: 
Return and Refund Policy Our Return and Refund Policy was last updated November 15th, 2024 Thank you for shopping at ECSS. If, for any reason, You are not completely satisfied with a purchase We invite You to review our policy on refunds and returns. This Return and Refund Policy was generated by TermsFeed Return and Refund Policy Template. The following terms are applicable for any products that You purchased with Us. Interpretation and Definitions Interpretation The words of which the initial letter is capitalized have meanings defined under the following conditions. The following definitions shall have the same meaning regardless of whether they appear in singular or in plural. Definitions For the purposes of this Return and Refund Policy:   "Company" (referred to as either "the Company", "We", "Us" or "Our" in this Agreement) refers to ECommerce Customer Service Squad (ECSS)   "Goods" refers to the items offered for sale on the Service.   "Orders" means a request b

### 2.5 Takeaways
* Vector search is a critical component of any LLM app using RAG architecture
* Cloudera's partner [Pinecone](https://www.pinecone.io/) provides a convenient SaaS offering for a Vector Database to support LLM RAG architecture
* Metadata from each entry in the Vector DB can be used to refine searches and add custom authorization frameworks

### Up Next: Go to Exercise 3
Where a gradio app is launched to complete the first iteration of the Q&A chat use case.